In [2]:
import utils.ss_transforms as sstr
from steps import Args
from steps import createServerStep1, createServerStep2, createServerStep3, createServerStep4

In [5]:
#some example
arg=Args(25,8,lr={'type':'polynomial','from':0.1,'to':0.05,'steps':45,'exponent':0.9})
arg=Args(25,8,lr={'type':'cyclical','min':0.05,'max':0.1,'period':46})
arg=Args(25,8,lr=0.05)
testTrStep1=sstr.Compose([sstr.ToTensor()])
trainTrStep1=sstr.Compose([sstr.ToTensor()])

In [6]:

#some example of functions
server1=createServerStep1(args=arg,train_transform=trainTrStep1,test_transform=testTrStep1)
server1.loadModel('MODEL_PATH') #to restore a saved model
#server1.train()
server1.test()
server1.eval_train()
server1.showClientSample('test_same_domain')

FileNotFoundError: [Errno 2] No such file or directory: 'data/idda/train.txt'

In [ ]:
#STEP 1 EX
arg=Args(8,25,lr=0.05,bs=4)
trainTr=sstr.Compose([sstr.RandomCrop(),sstr.Resize((540,960)),sstr.ToTensor()])
testTr=sstr.Compose([sstr.Resize((540,960)),sstr.ToTensor()])
server1=createServerStep1(args=arg,train_transform=trainTr,test_transform=testTr)
server1.train()

In [ ]:
#STEP 2 EX
arg=Args(num_rounds=80,num_epochs=4,lr=0.05,bs=4,clients_per_round=3,testEachRounds=5)
trainTr=sstr.Compose([sstr.RandomCrop(),sstr.Resize((540,960)),sstr.ToTensor()])
testTr=sstr.Compose([sstr.Resize((540,960)),sstr.ToTensor()])
server2=createServerStep2(args=arg,train_transform=trainTr,test_transform=testTr)
server2.train()

In [3]:
#STEP 3A EX
arg=Args(num_rounds=8,num_epochs=25,lr=0.05,bs=4,testEachRounds=5)
trainTr=sstr.Compose([sstr.RandomCrop(),sstr.Resize((540,960)),sstr.ToTensor()])
testTr=sstr.Compose([sstr.Resize((540,960)),sstr.ToTensor()])
server3=createServerStep3(args=arg,train_transform=trainTr,test_transform=testTr)
server3.train()

In [4]:
#STEP 3B EX
import utils.FDA as FDA
#In extractClientStyles must be an odd number >1.
#But in applyStyles it's enough that n is lower or equal (also =1) the tha one in extractClientStyles
n=5
size=(540,960)
styles=FDA.extractClientsStyles(n=n,size=size)
arg=Args(num_rounds=8,num_epochs=25,lr=0.05,bs=4,testEachRounds=5)
trainTr=sstr.Compose([sstr.RandomCrop(),sstr.Resize(size),sstr.ToTensor(),FDA.applyFDAstyles(styles=styles,n=n)])
testTr=sstr.Compose([sstr.Resize(size),sstr.ToTensor()])
server3=createServerStep3(args=arg,train_transform=trainTr,test_transform=testTr)
server3.train()

FileNotFoundError: [Errno 2] No such file or directory: 'data/idda/train.json'

In [ ]:
#STEP 4 EX
import torch
from models.deeplabv3 import deeplabv3_mobilenetv2

# Load best model
model = deeplabv3_mobilenetv2(16)
model.load_state_dict(torch.load("Fake_path.pt"))

# The teacher_model is updated every "teacher_update" rounds
# Unsupervised = True let the training phase be handled in Unsupervised settings
# (main changes: 
#                - initialization of teacher models, both in server and client
#                - different loss function in training phase that make use of pseudo-labels)
arg=Args(num_rounds=200,num_epochs=6,lr=0.05,bs=4,testEachRounds=1,
         clients_per_round=4, unsupervised=True, teacher_update=2)

trainTr=sstr.Compose([sstr.Resize((540,960)),sstr.ToTensor()])
testTr=sstr.Compose([sstr.Resize((540,960)),sstr.ToTensor()])
server4=createServerStep4(args=arg,train_transform=trainTr,test_transform=testTr, 
                          rootIdda="data/idda", model=model)
server4.train()